# Swarm Measurements

A short demo of taking magnetic measurements from a single Swarm spacecraft and deriving a main field model directly from one month of data!

Now that you have seen the principles of spherical harmonic models in the previous pages, here we use ChaosMagPy to provide the design matrix with which we will perform a simple least squares inversion using measurements from Swarm directly.

- [ChaosMagPy](https://chaosmagpy.readthedocs.io/), Clemens Kloss  
  https://doi.org/10.5281/zenodo.3352398
  - Provides the functions [design_gauss](https://chaosmagpy.readthedocs.io/en/master/functions/chaosmagpy.model_utils.design_gauss.html) and [synth_values](https://chaosmagpy.readthedocs.io/en/master/functions/chaosmagpy.model_utils.synth_values.html)
  
[hvPlot](https://hvplot.holoviz.org/) is used to create some fancy visualisations but this library is very complex to use! so if you are newer to the Python ecosystem, you are better off using Matplotlib for your own work.

In [ ]:
import datetime as dt
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import holoviews as hv
import hvplot.xarray

from viresclient import SwarmRequest
from chaosmagpy.model_utils import design_gauss, synth_values

## Fetching data to use

We fetch data as follows:
- `MAGA_LR` product from Swarm: this is the magnetic low rate (1Hz) data from Swarm Alpha
- the `B_NEC` variable is the field vector in the NEC (North, East, Centre) frame
- downsampled to one measurement per 10 seconds (`PT10S`)
- retaining only nominal measurements (according to `Flags_B` and `Flags_F`)
- during geomagnetically quiet times (Kp ≤ 2)
- during January 2020

In [ ]:
request = SwarmRequest()
request.set_collection("SW_OPER_MAGA_LR_1B")
request.set_products(
    measurements=["B_NEC"],
    sampling_step="PT10S",
)
request.set_range_filter("Flags_B", 0, 1)
request.set_range_filter("Flags_F", 0, 1)
request.set_range_filter("Kp", 0, 2)
data = request.get_between(
    start_time=dt.datetime(2020,1,1),
    end_time=dt.datetime(2020,2,1)
)
ds = data.as_xarray()
ds.attrs.pop("Sources")
ds

In [ ]:
_ds = ds.drop("Timestamp").sel(NEC="C")
_ds.hvplot.scatter(
    x="Longitude", y="Latitude", color="B_NEC",
    rasterize=True, colorbar=True, cmap="coolwarm", clabel="Vertical (downwards) magnetic field (nT)"
)

Above, we show the vertical (downwards) component of the magnetic field vector sampled across Earth. If you zoom in, you can see the lines curving as the orbits converge over the poles. As we have taken a whole month of data, the spatial coverage is quite dense.

## Building a model

Now we shall perform a simple inversion to fit the data, ($\vec{d}$), to the smooth spherical harmonic model, ($\vec{m}$). The relationship between $\vec{d}$ and $\vec{m}$ can be expressed as:

$$\vec{d} = G \vec{m}$$

where the $G$ is the "design matrix" which can be constructed by comparing to the spherical harmonic expansion of the magnetic field $\vec{B}$:

$$
\begin{align}\label{eq:spharm_B}
    B_i =\ & -\partial_i \left( R_E \sum\limits_{n=1}^{N} \sum\limits_{m=0}^{n} \left( g_n^m \cos{m\phi} + h_n^m \sin{m\phi}  \right)   \left(\frac{R_E}{r}\right)^{n+1} P_n^m (\cos{\theta}) \right) \nonumber \\
    =\ & -\partial_i \begin{bmatrix} 
R_E \sum\limits_{n=1}^{N} \sum\limits_{m=0}^{n} \left( \cos{m\phi}  \right)   \left(\frac{R_E}{r}\right)^{n+1} P_n^m (\cos{\theta}) \\
R_E \sum\limits_{n=1}^{N} \sum\limits_{m=0}^{n} \left( \sin{m\phi}  \right)   \left(\frac{R_E}{r}\right)^{n+1} P_n^m (\cos{\theta})
\end{bmatrix}
\begin{bmatrix}
g_n^m & h_n^m
\end{bmatrix}
\end{align}
$$

Note that:
- $B_i$ are the three vector components, $(B_r, B_\theta, B_\phi) = (-B_C, -B_N, B_E)$, i.e. the data, $\vec{d}$
- $\partial_i$ is a shorthand which should be replaced by the derivatives in spherical polar coordinates 
$
\left(  \partial_r = \frac{\partial}{\partial r}, \partial_\theta = \frac{1}{r} \frac{\partial}{\partial \theta}, \partial_\phi = \frac{1}{r\sin{\theta}} \frac{\partial}{\partial \phi} \right)
$
- $\begin{bmatrix} g_n^m & h_n^m \end{bmatrix}$ are the Gauss coefficients, i.e. the model, $\vec{m}$


A least-squares solution to $\vec{d} = G \vec{m}$ can be found with:

$$\vec{m} = (G^T G)^{-1} G^T \vec{d}$$

In [ ]:
def build_model(ds):
    """Use the contents of dataset, ds, to build a SH model"""
    # Get the positions and measurements from ds
    radius = (ds["Radius"]/1e3).values
    theta = (90-ds["Latitude"]).values
    phi = (ds["Longitude"]).values
    B_radius = -ds["B_NEC"].sel(NEC="C").values
    B_theta = -ds["B_NEC"].sel(NEC="N").values
    B_phi = ds["B_NEC"].sel(NEC="E").values
    # Use ChaosMagPy to build the design matrix, G
    # https://chaosmagpy.readthedocs.io/en/master/functions/chaosmagpy.model_utils.design_gauss.html
    G_radius, G_theta, G_phi = design_gauss(radius, theta, phi, nmax=13)
    # Using only the radial component, perform the inversion:
    G = G_radius
    d = B_radius
    m = np.linalg.inv(G.T @ G) @ (G.T @ d)
    return m

model_coeffs = build_model(ds)

In [ ]:
# The first 10 coefficients
model_coeffs[:10]

Let's sample our model and visualise it with contours:

In [ ]:
def sample_model_on_grid(m, radius=6371.200):
    """Evaluate Gauss coefficients over a regular grid over Earth"""
    theta = np.arange(1, 180, 1)
    phi = np.arange(-180, 180, 1)
    theta, phi = np.meshgrid(theta, phi)
    B_r, B_t, B_p = synth_values(m, radius, theta, phi)
    ds_grid = xr.Dataset(
        data_vars={"B_NEC_model": (("y", "x", "NEC"), np.stack((-B_t, B_p, -B_r), axis=2))},
        coords={"Latitude": (("y", "x"), 90-theta), "Longitude": (("y", "x"), phi), "NEC": np.array(["N", "E", "C"])}
    )
    return ds_grid

# Evaluate at the mean radius of the satellite measurements
mean_radius_km = float(ds["Radius"].mean())/1e3
ds_grid = sample_model_on_grid(model_coeffs, radius=mean_radius_km)
# Generate contour plot of vertical component
ds_grid.sel(NEC="C").hvplot.contourf(
    x="Longitude", y="Latitude", c="B_NEC_model",
    levels=30, coastline=True, projection=ccrs.PlateCarree(), global_extent=True,
    clabel="Model: vertical (downwards) magnetic field (nT)"
)

Nice! It looks like the main field: a curved magnetic equator dropping southwards over South America (around the South Atlantic Anomaly where the field is weaker), the South magnetic pole occurs over the edge of Antarctica towards Australia, the North magnetic pole is more extended over Northern Canada and Siberia. Though it is not as accurate as published models!

## Data-model residuals

Now let's dig a little deeper, comparing the input data and the model...

Let's plot the difference between the data and the model, the "data-model residuals":

In [ ]:
# Evaluate the model at the same points as the measurements
def append_model_evaluations(ds, m):
    ds = ds.copy()
    B_r, B_t, B_p = synth_values(m, ds["Radius"]/1e3, 90-ds["Latitude"], ds["Longitude"])
    ds["B_NEC_model"] = ("Timestamp", "NEC"), np.stack((-B_t, B_p, -B_r), axis=1)
    ds["B_NEC_res_model"] = ds["B_NEC"] - ds["B_NEC_model"]
    return ds
ds = append_model_evaluations(ds, model_coeffs)

# Plot the 
_ds = ds.drop("Timestamp").sel(NEC="C")
_ds.hvplot.scatter(
    x="Longitude", y="Latitude", color="B_NEC_res_model",
    rasterize=True, colorbar=True, cmap="coolwarm", clim=(-40, 40), clabel="Vertical (B_C) data-model residuals (nT)",
)

(Above): The difference between the model and the data is quite small except over the poles... let's look at the other components now:

In [ ]:
_ds = ds.drop("Timestamp").sel(NEC="E")
_ds.hvplot.scatter(
    x="Longitude", y="Latitude", color="B_NEC_res_model",
    rasterize=True, colorbar=True, cmap="coolwarm", clim=(-40, 40), clabel="Eastward (B_E) data-model residuals (nT)",
)

(Above): The disturbance over the poles is much stronger in the Eastward component...

In [ ]:
ds.drop("Timestamp").hvplot.scatter(x="Latitude", y="B_NEC_res_model", col="NEC", rasterize=True, colorbar=False)

(Above): Both the Northward (N) and Eastward (E) components are more disturbed over the poles than the downward (C) component. This disturbance is due to Field-Aligned Currents (FACs) and Auroral Electrojets (AEJs) which produce strong magnetic fields over the poles.

In [ ]:
fig, ax = plt.subplots(1, 1)
(ds.groupby_bins("Latitude", 90)
   .apply(lambda x: x["B_NEC_res_model"].std(axis=0))
   .plot.line(x="Latitude_bins", ax=ax)
)
ax.set_title("Standard deviations");

## Exploring further

Take a look at [Swarm Notebooks](https://swarm.magneticearth.org/notebooks/04a1_geomag-models-vires) if you want more examples. There you will also find recipes for accessing other Swarm products.